In [1]:
import torch

In [2]:
def calc_eer_for_binary_classification(targets_scores: torch.Tensor, imposter_scores: torch.Tensor, iters: int = 100) -> float:
    """
    Computes the Equal Error Rate (EER) for a binary classification system.

    The EER is the point where the False Acceptance Rate (FAR) and the False Rejection Rate (FRR) are equal.

    :param targets_scores: torch.Tensor, scores assigned to genuine (target) samples.
    :param imposter_scores: torch.Tensor, scores assigned to imposter samples.
    :param iters: int, number of threshold iterations to evaluate EER.
    :return: float, the computed Equal Error Rate (EER).

    Errors:
    - Returns NaN if either targets_scores or imposter_scores is empty.

    The function iterates over `iters` threshold values, computes FAR and FRR at each step,
    and determines the threshold where these rates are closest to being equal. The EER is the
    average of FAR and FRR at this threshold.
    """

    if len(targets_scores) == 0 or len(imposter_scores) == 0:
        return float('nan')
    min_score = torch.min(targets_scores.min(), imposter_scores.min())
    max_score = torch.max(targets_scores.max(), imposter_scores.max())

    n_tars = targets_scores.numel()
    n_imps = imposter_scores.numel()

    dists = torch.linspace(min_score, max_score, iters)
    fars = torch.zeros(iters, dtype=torch.float32)
    frrs = torch.zeros(iters, dtype=torch.float32)

    min_diff = float('inf')
    eer = 0

    for i, dist in enumerate(dists):
        far = (imposter_scores >= dist).sum().float() / n_imps
        frr = (targets_scores <= dist).sum().float() / n_tars
        fars[i] = far
        frrs[i] = frr

        diff = torch.abs(far - frr)
        if diff < min_diff:
            min_diff = diff
            eer = (far + frr) / 2

    # return eer.item(), fars, frrs, dists
    return eer.item()

In [3]:
def calc_eer(target: torch.Tensor, predicted: torch.Tensor) -> torch.Tensor:
    """
    Computes the Equal Error Rate (EER) for a multi-class classification problem using PyTorch tensors.

    The function calculates EER for each class separately by treating it as a binary classification problem.
    It then returns a tensor containing the EER for each class.

    :param target: torch.Tensor, a 1D tensor (N,) containing the true class labels.
    :param predicted: torch.Tensor, a 2D tensor (N, C), where N is the number of samples and C is the number of classes.
                      Each row contains the predicted scores or probabilities for each class.
    :return: torch.Tensor, a 1D tensor (C,) containing the EER for each class.

    The function iterates over all classes, treating each as the positive class and the rest as negative.
    It then computes EER using `calc_eer_for_binary_classification` and returns the EER values for each class.
    """

    num_classes = predicted.shape[1]
    all_indices = torch.arange(num_classes)
    eer = torch.zeros((num_classes,))
    for cl in range(num_classes):
        fixed_indices = torch.where(target == cl)[0]
        other_indices = torch.where(target != cl)[0]
        fixed = predicted[fixed_indices, cl]
        other = predicted[other_indices, cl]
        eer[cl] = calc_eer_for_binary_classification(fixed, other)

    # return torch.nanmean(eer).item()
    return eer

<p>Здесь должна быть довольно маленькая ошибка, потому что классы хорошо отличимы</p>

In [4]:
y = torch.Tensor([0, 1, 0, 1, 2, 3])
y_pred = torch.Tensor([
    [0.6, 0.1, 0.1, 0.2],
    [0.1, 0.5, 0.2, 0.2],
    [0.7, 0.1, 0.1, 0.1],
    [0.1, 0.6, 0.2, 0.1],
    [0.1, 0.1, 0.6, 0.2],
    [0.1, 0.1, 0.1, 0.7],
])
calc_eer(y, y_pred)

tensor([0., 0., 0., 0.])

<p>Здесь должна быть довольно большая ошибка, потому что классы плохо отличимы</p>

In [5]:
y = torch.Tensor([0, 1, 0, 1, 2, 3])
y_pred = torch.Tensor([
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25],
])
calc_eer(y, y_pred)

tensor([1., 1., 1., 1.])

<p>И снова большая ошибка</p>

In [7]:
y = torch.Tensor([0, 1, 0, 1, 3, 3])
y_pred = torch.nn.functional.softmax(torch.rand(y.shape[0], torch.max(y).int().item() + 1), dim=1)
print(y_pred)
calc_eer(y, y_pred)

tensor([[0.3462, 0.1987, 0.2687, 0.1864],
        [0.3216, 0.1729, 0.2477, 0.2579],
        [0.3066, 0.2652, 0.2560, 0.1722],
        [0.2309, 0.3243, 0.1746, 0.2702],
        [0.2750, 0.2570, 0.2410, 0.2270],
        [0.3142, 0.3290, 0.1721, 0.1847]])


tensor([0.5000, 0.5000,    nan, 0.5000])